In [ ]:
"""
Generate positive and negative dataset.
Still not expanded.
"""
from data_preparation import generate_csv_from_fasta, generate_negative_dataset
from data_dir import epd_tata, epd_pos_tata_csv, epd_neg_tata_csv

print("Generate positive csv from EPD data {}".format(generate_csv_from_fasta(epd_tata, epd_pos_tata_csv, 1)))
print("Generate negative csv from positive data {}".format(generate_negative_dataset(epd_pos_tata_csv, epd_neg_tata_csv)))

In [ ]:
"""
Generate kmer version for both positive and negative promoter data.
"""
from data_preparation import generate_kmer_csv
from data_dir import epd_pos_tata_csv, epd_neg_tata_csv, epd_pos_tata_kmer_csv, epd_neg_tata_kmer_csv

print("Generate kmer csv from positive data {}".format(generate_kmer_csv(epd_pos_tata_csv, epd_pos_tata_kmer_csv)))
print("Generate kmer csv from negative data {}".format(generate_kmer_csv(epd_neg_tata_csv, epd_neg_tata_kmer_csv)))

In [1]:
"""
Generate train, validation, and test data for both positive and negative promoter data.
"""
from data_preparation import generate_datasets
from data_dir import epd_pos_tata_kmer_csv, epd_neg_tata_kmer_csv, epd_pos_tata_kmer_dir, epd_neg_tata_kmer_dir

generate_datasets(epd_pos_tata_kmer_csv, epd_pos_tata_kmer_dir)
generate_datasets(epd_neg_tata_kmer_csv, epd_neg_tata_kmer_dir)

['./data/epd/human_non_tata_kmer/train.csv',
 './data/epd/human_non_tata_kmer/validation.csv',
 './data/epd/human_non_tata_kmer/test.csv']

In [1]:
"""
Generate expanded datasets.
"""
import os
from data_dir import epd_pos_tata_kmer_dir, epd_neg_tata_kmer_dir
from data_preparation import expand_files_in_dir

print("Expanding positive dataset: {}".format(expand_files_in_dir(epd_pos_tata_kmer_dir)))
print("Expanding negative dataset: {}".format(expand_files_in_dir(epd_neg_tata_kmer_dir)))

Expanding positive dataset: Truean_tata_kmer/validation.csv: 207/207
Expanding negative dataset: Truean_non_tata_kmer/validation.csv: 207/207


In [1]:
"""
Generate samples for train, validation, and data for both positive and negative dataset.
"""
from data_preparation import generate_sample
from data_dir import epd_pos_tata_kmer_dir, epd_neg_tata_kmer_dir, sample_prom_pos_dir, sample_prom_neg_dir

# Create samples.
prom_pos_srcs = [
    ("{}/train.csv".format(epd_pos_tata_kmer_dir), "{}/train.csv".format(sample_prom_pos_dir)),
    ("{}/validation.csv".format(epd_pos_tata_kmer_dir), "{}/validation.csv".format(sample_prom_pos_dir)),
    ("{}/test.csv".format(epd_pos_tata_kmer_dir), "{}/test.csv".format(sample_prom_pos_dir) ),
]
prom_neg_srcs = [
    ("{}/train.csv".format(epd_neg_tata_kmer_dir), "{}/train.csv".format(sample_prom_neg_dir)), 
    ("{}/validation.csv".format(epd_neg_tata_kmer_dir), "{}/validation.csv".format(sample_prom_neg_dir)),
    ("{}/test.csv".format(epd_neg_tata_kmer_dir), "{}/test.csv".format(sample_prom_neg_dir)),
]
domain = [prom_pos_srcs, prom_neg_srcs]
for dom in domain:
    for src in dom:
        src_csv = src[0]
        target_csv = src[1]
        generate_sample(src_csv, target_csv, frac_sample=0.1)

In [2]:
"""
Merge samples.
"""
from data_dir import sample_prom_neg_dir, sample_prom_pos_dir, sample_prom_train_sample_csv, sample_prom_validation_sample_csv, sample_prom_test_sample_csv
from data_preparation import merge_csv

print("Merging sample training data: {}".format(merge_csv([
    "{}/train.csv".format(sample_prom_pos_dir),
    "{}/train.csv".format(sample_prom_neg_dir)
], sample_prom_train_sample_csv)))
print("Merging sample validation data: {}".format(merge_csv([
    "{}/validation.csv".format(sample_prom_pos_dir),
    "{}/validation.csv".format(sample_prom_neg_dir)
], sample_prom_validation_sample_csv)))
print("Merging sample test data: {}".format(merge_csv([
    "{}/test.csv".format(sample_prom_pos_dir),
    "{}/test.csv".format(sample_prom_neg_dir)
], sample_prom_test_sample_csv)))

Merging sample training data: True
Merging sample validation data: True
Merging sample test data: True


In [1]:
"""
Merge original data.
"""
from data_dir import epd_pos_tata_train_csv, epd_pos_tata_validation_csv, epd_pos_tata_test_csv, epd_neg_tata_train_csv, epd_neg_tata_validation_csv, epd_neg_tata_test_csv
from data_dir import dataset_prom_train_csv, dataset_prom_validation_csv, dataset_prom_test_csv
from data_preparation import merge_csv

print("Merging training data: {}".format(merge_csv([
    epd_pos_tata_train_csv,
    epd_neg_tata_train_csv,
], dataset_prom_train_csv)))
print("Merging validation data: {}".format(merge_csv([
    epd_pos_tata_validation_csv,
    epd_neg_tata_validation_csv
], dataset_prom_validation_csv)))
print("Merging test data: {}".format(merge_csv([
    epd_pos_tata_test_csv,
    epd_neg_tata_test_csv
], dataset_prom_test_csv)))

Merging training data: True
Merging validation data: True
Merging test data: True


In [1]:
"""
Expand train, validation, and test data into 510 tokens for each sequence and store the sequence into dataset folder.
Since these are not samples then use `./dataset/full/promoter` folder.
"""
from data_dir import dataset_full_prom_dir, data_epd_dir, _generic_filenames
from data_preparation import expand_by_sliding_window_no_pandas

_files = _generic_filenames
for fname in _files:
    src_path = "{}/{}".format(data_epd_dir, fname)
    target_path = "{}/{}".format(dataset_full_prom_dir, fname)
    print("Expanding {} => {}: {}".format(src_path, target_path, expand_by_sliding_window_no_pandas(src_path, target_path, length=510)))



Expanding ./data/epd/train.csv => ./dataset/full/promoter/train.csv: True
Expanding ./data/epd/validation.csv => ./dataset/full/promoter/validation.csv: True
Expanding ./data/epd/test.csv => ./dataset/full/promoter/test.csv: True


In [2]:
from data_dir import epd_pos_tata_csv, epd_neg_tata_csv
import pandas as pd

epd_pos_df = pd.read_csv(epd_pos_tata_csv)
print(len(epd_pos_df))
epd_neg_df = pd.read_csv(epd_neg_tata_csv)
print(len(epd_neg_df))

2067
2067


In [1]:
"""
Merging Human TATA and Human Non TATA csv into single file.
"""
from data_dir import epd_neg_tata_kmer_csv, epd_pos_tata_kmer_csv, epd_train_csv
from data_preparation import merge_csv

files = [epd_neg_tata_kmer_csv, epd_pos_tata_kmer_csv]
print("Merging positive and negative TATA promoter {} => {}: {}".format(files, epd_train_csv, merge_csv(files, epd_train_csv)))


Merging positive and negative TATA promoter ['./data/epd/human_non_tata_kmer.csv', './data/epd/human_tata_kmer.csv'] => ./data/epd/train.csv: True
